In [981]:
import random

At first we generate our Population

In [982]:
def generation_poupulation(nombre_individu):
    population = []
    for _ in range(nombre_individu):
        individu = [random.randint(0,1) for _ in range(8)]
        population.append(individu)
    return population

Pop = generation_poupulation(30)
print(Pop)

[[1, 1, 1, 0, 0, 0, 0, 1], [1, 0, 0, 1, 1, 1, 1, 0], [0, 0, 0, 1, 1, 1, 0, 0], [1, 0, 1, 1, 1, 0, 1, 0], [0, 1, 0, 0, 1, 0, 0, 0], [1, 0, 0, 0, 1, 0, 0, 0], [1, 1, 1, 1, 0, 1, 1, 0], [0, 0, 1, 0, 0, 0, 1, 1], [1, 1, 0, 1, 0, 1, 1, 0], [1, 0, 1, 1, 0, 0, 1, 0], [0, 0, 1, 1, 0, 1, 0, 1], [1, 1, 0, 0, 1, 0, 0, 1], [0, 1, 1, 0, 1, 0, 1, 0], [0, 0, 1, 1, 1, 0, 1, 1], [1, 1, 0, 1, 1, 1, 1, 1], [1, 1, 1, 0, 0, 0, 0, 0], [1, 1, 0, 0, 0, 1, 0, 0], [0, 1, 0, 1, 0, 0, 1, 0], [1, 1, 1, 0, 1, 0, 0, 1], [0, 1, 1, 1, 0, 0, 0, 1], [1, 0, 1, 1, 0, 0, 1, 1], [1, 0, 1, 1, 1, 0, 1, 1], [0, 0, 0, 1, 1, 1, 0, 0], [1, 1, 0, 1, 0, 0, 0, 0], [1, 1, 1, 1, 0, 0, 1, 0], [0, 0, 1, 1, 0, 1, 1, 0], [0, 0, 1, 1, 0, 0, 0, 1], [0, 0, 1, 1, 0, 0, 1, 1], [1, 1, 1, 0, 1, 1, 1, 0], [0, 0, 0, 1, 1, 1, 1, 0]]


Then we evaluate each individual of our population by counting the number of 1 they have

In [983]:
def evaluate(Population):
    score = []
    for candidate in Population:
        score.append(candidate.count(1))
    return score

# print(evaluate(generation_poupulation()))
# evaluate(Pop)

This is our tournament selection function in which we take : k members chosen randomly. It returns a list of our chosen candidates

In [984]:
def tournament(k, population):
    """k est la taille du tournoi"""
    tournament_selection = []
    for _ in range(k):
        chosen = random.choice(population)
        tournament_selection.append(chosen)
    return tournament_selection


aga = tournament(3, generation_poupulation(20))
print(aga)

[[1, 0, 1, 1, 0, 1, 1, 1], [1, 1, 0, 1, 1, 1, 0, 1], [1, 1, 1, 1, 1, 1, 0, 1]]


The roulette_wheel function take as parameter a population and returns a candidate (list of 0 and 1) that is chosen. The highest number of one the candidate have, the highest probability it has to be selected.

In [985]:
def roulette_wheel(Population):
    wheel = []
    for candidate in Population:
        x = candidate.count(1)
        match x:
            case 8:
                for _ in range(5):
                    wheel.append(candidate)
            case 7:
                for _ in range(4):
                    wheel.append(candidate)
            case 6:
                for _ in range(4):
                    wheel.append(candidate)
            case 5:
                for _ in range(3):
                    wheel.append(candidate)
            case 4:
                for _ in range(3):
                    wheel.append(candidate)
            case 3:
                for _ in range(2):
                    wheel.append(candidate)
            case 2:
                for _ in range(2):
                    wheel.append(candidate)
            case 1:
                for _ in range(1):
                    wheel.append(candidate)
            case 0:
                for _ in range(1):
                    wheel.append(candidate)
            case _:
                print("Error")
    choice = random.choice(wheel)
    return choice
# rw = roulette_wheel(Pop)
# print(rw)

This function works in peer with the tournament function, it evaluates among the candidates in the tournament which is the best

In [986]:
def find_the_best(array):
    best_element = 0
    best_candidate = None
    for index, candidate in enumerate(array):
        if sum(candidate)>best_element:
            best_candidate = index
    return array[best_candidate]

#print(find_the_best(aga))

Here we select the parents of the next generation, two mode available

In [987]:
def selection_parent(population, mode):
    best_parent = []
    if mode == "tournament":
        for _ in range(len(population)):
            k = 3
            candidate = find_the_best(tournament(k,population))
            if best_parent.count(candidate) > 3:
                candidate1 = find_the_best(tournament(k,population))
            best_parent.append(find_the_best(candidate1))

    if mode == "roulette_wheel":
        for _ in range(len(population)):
                best_candidate1 = roulette_wheel(population)
                if best_parent.count(best_candidate1) > 3:
                    best_candidate = roulette_wheel(population)
                    if best_candidate != best_candidate1:
                        best_parent.append(best_candidate)
                # ici on évite de prendre le meme candidat pour constituer toute notre population car sinon à partir d'une certaine génaration, le meilleur ne vas se repoduire qu'avec lui même
                best_parent.append(best_candidate1)

    return best_parent

# print(selection_parent(Pop))
# print(len(selection_parent(Pop)))



We generate couples (list of two random parents) inside a couple list

In [988]:
def couple_selection(the_parents, nombre_individu):
    couple_list = []
    for _ in range(int(nombre_individu/2)):
        couple_list.append([random.choice(the_parents),random.choice(the_parents)])
    return couple_list

# print(couple_selection())




Here we do a crossover between the two parents, echanging "genes" of the middle like [0,0,1,1] and [1,1,0,0] becomes [0,1,0,1] and [1,0,1,0] so it's the child

In [989]:
def child_creation(couples):
    #ici les couples vont passé de couple parent à couple enfant
    for couple in couples:
        # effectue le crossover entre un intervalle de 2 et de 6 digits centraux de chaque parent
        crossover = random.randint(1,6)
        temp = couple[1][crossover:-crossover]
        couple[1][crossover:-crossover] = couple[0][crossover:-crossover]
        couple[0][crossover:-crossover] = temp
    return couples #retourne les couples enfant

# child_creation(couple_selection())

For each childrens, they have a chance, here equal to 0.005 to mutate, input : liste of children ; output : list of muted children

In [990]:
def random_mutation(couples):
    muted_children = []
    for duo in couples:
        for child in duo:
            if random.random()>0.995:
                number = random.choice([0,1,2,3,4,5,6,7])
                if child[number] == 0:
                    child[number] = 1
                else:
                    child[number] = 1
            muted_children.append(child)
    return muted_children


# print(random_mutation(child_creation(couple_selection())))
# evaluate(random_mutation(child_creation(couple_selection())))

Here our main code that process all the steps

In [991]:
def main():
    nombre_individu = 8
    iterations = 1000
    #Generation initiale de la population
    population = generation_poupulation(nombre_individu)
    print(population)
    print("Depart")
    #evaluation des parents
    # evaluation_parent = evaluate(population)
    # print(evaluation_parent)

    #les lignes suivantes : processus de création d'une nouvelle population
    for it in range(iterations):
        # candidate_tourniquet = tournament(3, population)
        # best = find_the_best(candidate_tourniquet)
        the_parents = selection_parent(population, "tournament")
        selection_couple =couple_selection(the_parents, nombre_individu)
        the_children = child_creation(selection_couple)
        #mutation_childrens est le groupe d'enfant qui constitue la nouvelle population
        mutation_children = random_mutation(the_children)
        population = mutation_children
        print(population)
        if [1, 1, 1, 1, 1, 1, 1, 1] in mutation_children:
            print(f"MaxOne à l'itération {it}")
            break

    if [1, 1, 1, 1, 1, 1, 1, 1] not in population:
        print(f"pas de max0ne")

    print(population)


    return None

main()

[[0, 1, 0, 1, 0, 0, 0, 0], [1, 1, 1, 0, 1, 0, 0, 0], [0, 1, 0, 1, 0, 0, 0, 1], [0, 0, 1, 1, 0, 1, 1, 0], [0, 0, 1, 0, 0, 0, 1, 0], [1, 1, 1, 1, 1, 0, 1, 1], [1, 1, 1, 0, 1, 1, 0, 1], [1, 0, 1, 1, 1, 1, 0, 0]]
Depart
[[0, 1, 0, 1, 0, 0, 0, 1], [1, 0, 1, 1, 1, 1, 0, 0], [1, 0, 1, 1, 1, 1, 0, 0], [1, 0, 1, 1, 1, 1, 0, 0], [1, 1, 1, 1, 1, 0, 1, 1], [0, 1, 0, 0, 1, 0, 0, 0], [0, 1, 0, 0, 1, 0, 0, 0], [1, 1, 1, 1, 0, 1, 0, 1]]
[[0, 1, 0, 0, 1, 0, 0, 0], [1, 1, 1, 1, 1, 1, 0, 1], [1, 0, 1, 1, 0, 1, 0, 0], [1, 0, 1, 1, 0, 1, 0, 0], [1, 0, 1, 1, 0, 1, 0, 0], [1, 0, 1, 1, 0, 1, 0, 0], [1, 0, 1, 1, 0, 1, 0, 0], [1, 1, 1, 1, 1, 1, 0, 1]]
[[1, 1, 0, 0, 1, 0, 0, 0], [1, 1, 0, 0, 1, 0, 0, 0], [1, 1, 0, 0, 1, 0, 0, 0], [1, 1, 0, 0, 1, 0, 0, 0], [1, 1, 0, 0, 1, 0, 0, 0], [0, 0, 1, 1, 0, 1, 0, 0], [1, 1, 0, 0, 1, 0, 0, 0], [1, 1, 0, 0, 1, 0, 0, 0]]
[[1, 1, 0, 0, 1, 0, 0, 0], [1, 1, 0, 0, 1, 0, 0, 0], [1, 1, 0, 0, 1, 0, 0, 0], [1, 1, 0, 0, 1, 0, 0, 0], [1, 1, 0, 0, 1, 0, 0, 0], [1, 1, 0, 0, 1, 0, 0, 0], 

Travail sur un Genetic algorithm
